Google Colab

In [2]:
import torch

print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

Torch version:1.10.0+cu111
cuda version: 11.1
cudnn version:8005


In [3]:
import time
from numpy import zeros, roll


grid_shape = (640, 640)


def laplacian(grid):
    return (roll(grid, +1, 0) +
            roll(grid, -1, 0) +
            roll(grid, +1, 1) +
            roll(grid, -1, 1) -
            4 * grid)


def evolve(grid, dt, D=1):
    return grid + dt * D * laplacian(grid)


def run_experiment(num_iterations):
    grid = zeros(grid_shape)

    block_low = int(grid_shape[0] * 0.4)
    block_high = int(grid_shape[0] * 0.5)
    grid[block_low:block_high, block_low:block_high] = 0.005

    start = time.time()
    for i in range(num_iterations):
        grid = evolve(grid, 0, 1)
    return time.time() - start


if __name__ == "__main__":
    print(f"elapsed time: {run_experiment(500)}")


elapsed time: 2.3353774547576904


In [4]:
import time
from torch import zeros, roll


grid_shape = (640, 640)


def laplacian(grid):
    return (roll(grid, +1, 0) +
            roll(grid, -1, 0) +
            roll(grid, +1, 1) +
            roll(grid, -1, 1) -
            4 * grid)


def evolve(grid, dt, D=1):
    return grid + dt * D * laplacian(grid)


def run_experiment(num_iterations):
    grid = zeros(grid_shape)

    block_low = int(grid_shape[0] * 0.4)
    block_high = int(grid_shape[0] * 0.5)
    grid[block_low:block_high, block_low:block_high] = 0.005

    grid = grid.cuda()
    start = time.time()
    for i in range(num_iterations):
        grid = evolve(grid, 0, 1)
    return time.time() - start


if __name__ == "__main__":
    print(f"elapsed time: {run_experiment(500)}")


elapsed time: 0.21639752388000488


In [1]:
!nvidia-smi

Sat Dec 11 15:37:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------